In [ ]:
!pip install cleantext better_profanity
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install poetry-version
!pip install cached-property
!pip install redis
!pip install ordered_set
!pip install ruamel.yaml
!pip install ordered-set-stubs
!pip install profanity-filter --no-deps

In [33]:
import re
from cleantext import clean
from better_profanity import profanity
import requests
import spacy
from spacy.tokens import Token
from profanity_filter import ProfanityFilter

file_name = 'obscene_lexicon.txt'
with open(file_name, 'r', encoding='utf-8') as file:
    bad_words = file.read().split('\n')

nlp = spacy.load("ru_core_news_sm")
Token.set_extension("is_profane", default=False, force=True)
pf = ProfanityFilter(languages=['ru'], nlps={'ru': nlp})

def clean_answer(text):
    def remove_code_from_text(text):
        code_pattern = re.compile(r'\b(def|class|if|else|elif|for|while|try|except|finally|with|import|from|print|return|break|continue|pass|raise|assert|yield|async|await|lambda|global|nonlocal|del|exec|eval)\b.*', re.DOTALL)
        fenced_code_pattern = re.compile(r'```.*?```', re.DOTALL)
        cleaned_text = code_pattern.sub('', text)
        cleaned_text = fenced_code_pattern.sub('', cleaned_text)
        return cleaned_text

    def remove_obscene_lexicon(text):
        cleaned_text = clean(text, extra_spaces=True, stemming=False, stopwords=True, lowercase=False, stp_lang='english' )
        cleaned_text = profanity.censor(cleaned_text)
        cleaned_text = pf.censor(cleaned_text)
        for word in bad_words:
            # cleaned_text = cleaned_text.replace(word, '***')
            bad_words_re = re.compile(r'\b(' + word + r')\b', re.IGNORECASE)
            cleaned_text = bad_words_re.sub(lambda match: '***', cleaned_text)
        # print(bad_words_re)
        # cleaned_text = bad_words_re.sub(lambda match: '*' * len(match.group()), cleaned_text)
        return cleaned_text

    text = remove_code_from_text(text)
    text = remove_obscene_lexicon(text)
    return text

In [34]:
text = """
Можно решить задачу так, проститутка, bitch, хуй:

def remove_empty_lines(text):
    return "\n".join([line for line in text.splitlines() if line.strip()])

def create_request(row):
    # Удаляем пустые строки в описании задачи
    description = remove_empty_lines(row['description'])
    req = f"Есть задача, описанная так: ${description}$."

    # Удаляем пустые строки в решении студента
    student_solution = remove_empty_lines(row['student_solution'])
    if student_solution:
        req += f" У меня есть решение: ${student_solution}$."

    if row["code_problem"]:
        # Удаляем пустые строки в сообщении об ошибке
        problem_message = remove_empty_lines(row['problem_message'])
        if problem_message:
            req += f" В коде есть ошибка: ${problem_message}$."
    else:
        if row["test_info"]:
            req += " Оно не проходит тесты."
            if len(row["test_info"]) > 0:
                test = row["test_info"][0]
                if len(test) >= 3 and test[0] and test[1] and test[2]:
                    req += f" При вводе {test[0]} ожидалось {test[1]}, но я получаю {test[2]}.\n"

    req += " Дай короткую подсказку в одном предложении, что нужно сделать, чтобы исправить задачу."

    return req.replace('\n', '')
"""

clean_answer(text)

'Можно решить задачу так, ***, ****, ***:'